In [1]:
import syft as sy
from syft.core.node.new.credentials import SyftVerifyKey
from typing import Dict
from typing import Any
import torch as th

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft
/home/teo/anaconda3/envs/PySyft/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-31 17:21:13.366749: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64:/usr/local/cuda-11.5/lib64:
2023-01-31 17:21:13.381185: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-31 17:21:13.872889: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Cou

In [2]:
ds_client = sy.login(email="info@openmined.org", password="changethis", port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into test... done!


In [3]:
def foo():
    print("Hello")

In [4]:
class Policy():
    
    request_itereations: int
    my_verify_key: SyftVerifyKey
    state: Dict[str, Any] = None

    def public_state(self):
        return NotImplemented
        
    def start(self):
        return NotImplemented
        
    def apply(self, kwargs, state):
        return NotImplemented

class ArgBinderSubPolicy:
    
    def __init__(self, binding_kwargs):
        self.binding_kwargs = binding_kwargs
        
    def kargs_are_valid(self, *kwargs):
        
        looks_good = True
        
        # insert logic here which changes looks_good to
        # false if the *kwargs do NOT match the 
        # limitations in binding_kwargs
        
        return looks_good
    
class RepeatedCallPolicyWithBoundArgs(Policy):
        
    def __init__(self, n_calls, downloadable_output_args, **binding_kwargs):
        self.n_calls = n_calls
        self.binding_policy = ArgBinderSubPolicy(binding_kwargs)
        self.downloadable_output_args = downloadable_output_args
        
    def start(self):# -> None
        self.state = {"count": 0}
    
    def public_state(self):
        return self.state
        
    def apply_output(self, results_dict, **kwargs) -> Dict[Any, Any]:
        
        if self.binding_policy.kwargs_are_valid(kwargs):

            if self.state["count"] < self.n_calls:
                for output_arg in self.downloadable_output_args:
                    results_dict[output_arg] = results_dict[output_arg].set_permission("READ", self.my_verify_key)

            self.state["count"] += 1
            
        return results_dict
    
class SingleCallPolicyWithBoundArgs(RepeatedCallPolicyWithBoundArgs):
    
    def __init__(self, downloadable_output_args, **binding_kwargs):
        self.n_calls = 1
        self.binding_policy = ArgBinderSubPolicy(binding_kwargs)
        self.downloadable_output_args = downloadable_output_args
    

In [9]:
f_string = """y = x + x*2"""

In [15]:
x = th.tensor([1,2,3,4,5,6,7]).send(ds_client)

In [16]:
x_pointer = ds_client.store[x.id_at_location.to_string()]

In [20]:
x_pointer.get_copy()

tensor([1, 2, 3, 4, 5, 6, 7])

In [39]:
ds_client.code_request(code=f_string, inputs={"x": x}, outputs=['y'])

In [45]:
task_f_string = ds_client.tasks["tasks"][-1]["uid"]
task_f_string

'4f04955c888843f6ac2ad1e731636346'

In [43]:
[task['uid'] for task in ds_client.tasks["tasks"]]

['cbae8b176c8b48fc83fef3f72a58e9f0',
 'a2c55ac7d30d42229a3eff49131894ea',
 'afe6184d082941c8bbd711a7e22ff12f',
 '9d374c4ac9f04f1091cee1249343dbf2',
 '022809b051ac4baaa48ddf108b297139',
 'c29fa58746e1401c8631633350048a0a',
 '4f04955c888843f6ac2ad1e731636346']

In [48]:
ds_client.tasks["tasks"][-1]

{'id': <UID: 476b1254f58b45f58077130ab187c73c>,
 'uid': '4f04955c888843f6ac2ad1e731636346',
 'user': 'a9e91773e79d4620866e5350e534b63e',
 'inputs': {'x': '0744e6393089495abbb15e9147159fad'},
 'owner': {'name': 'Jane Doe', 'role': 'Owner', 'email': 'info@openmined.org'},
 'code': 'y = x + x*2',
 'status': 'accepted',
 'created_at': '31/01/2023',
 'updated_at': '31/01/2023',
 'reviewed_by': 'Jane Doe',
 'execution': {'status': 'done'},
 'outputs': {'output': 'df08a7dfbd6f4e7696186611bd49b253'},
 'reason': 'Looks good'}

In [ ]:
type(task_f_string.to_string())

str

In [47]:
ds_client.review(task_uid=task_f_string, approve=True, reason="Looks good")

4f04955c888843f6ac2ad1e731636346


In [57]:
y_uid = ds_client.tasks["tasks"][-1]["outputs"]["output"]

In [58]:
y_pointer = ds_client.store.get(y_uid)

In [59]:
y_pointer.get()

{'y': tensor([ 3,  6,  9, 12, 15, 18, 21])}